<a href="https://colab.research.google.com/github/chicanoandres702/AI-File-Organizer/blob/main/GPT_SoVITS_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-SoVITS-WebUI
A Few-shot & Zero-shot Text-to-Speech WebUI.


[General GPT-SoVITS Guide](https://rentry.co/GPT-SoVITS-guide#3-initialize-dataset)

### Credits:

- Original Project & Colab done by [GPT-SoVITS Team](https://github.com/RVC-Boss/GPT-SoVITS/graphs/contributors)

- Port made by [Nick088](https://linktr.ee/Nick088)

- Testing/Tweaks done by [vulkanitexd](https://discord.com/users/984567398826917918)

In [1]:
#@title Prepare the Environment
#@markdown It git clones the repository.

# get the repo
!git clone https://github.com/RVC-Boss/GPT-SoVITS
%cd GPT-SoVITS

Cloning into 'GPT-SoVITS'...
remote: Enumerating objects: 4172, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4172 (delta 5), reused 5 (delta 5), pack-reused 4164 (from 3)
Receiving objects: 100% (4172/4172), 12.43 MiB | 14.92 MiB/s, done.
Resolving deltas: 100% (2425/2425), done.
/content/GPT-SoVITS


In [2]:
#@title (OPTIONAL) Mount Google Drive

#@markdown This will create a `GPT-SoVITS` folder in [your Google Drive](https://drive.google.com/drive/u/0/home), that can be used to put things like your model dataset, but you have to **manually put the full path** which can be `/content/drive/MyDrive/GPT-SoVITS/YourDatasetHere`
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the parent directory path
parent_dir = '/content/drive/MyDrive/GPT-SoVITS'

# Create the parent directory if it doesn't exist
if not os.path.exists(parent_dir):
    os.mkdir(parent_dir)

Mounted at /content/drive


In [3]:
#@title Install Conda
#@markdown Conda will make the session crash/restart, it's normal, just run the following cells after.
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://repo.anaconda.com/miniconda/Miniconda3-py39_24.11.1-0-Linux-x86_64.sh")

⏬ Downloading https://repo.anaconda.com/miniconda/Miniconda3-py39_24.11.1-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:23
🔁 Restarting kernel...


In [ ]:
#@title Install requirements
#@markdown Will take alot of time..

#@markdown The runtime will restart itself, it's normal.

from IPython.display import clear_output
import time

%cd /content/GPT-SoVITS/
!bash install.sh

# fix packages asr models japanese/english (whisper)
!pip install ctranslate2==4.4.0

# fix https://github.com/RVC-Boss/GPT-SoVITS/issues/1715
import nltk
nltk.download('averaged_perceptron_tagger_eng')

# fix for issue in inference
!pip install -q ipykernel

# fix train gpt
!pip install torchmetrics==1.5

clear_output()

print("Installed the requirements!")

time.sleep(5)
# restart runtime
import os
os._exit(0)

Installed the requirements!


In [ ]:
#@title Download Pretrained Models
#@markdown gpt so vits v1 & v2 pretrains + uvr5 weights + chinese asr. (english/japanese asr, faster whisper, should be automatically installed when using it for ASR in the UI)

from IPython.display import clear_output


%cd /content/


# gpt so vits pretrains

!git clone https://huggingface.co/lj1995/GPT-SoVITS/ gptsovits_pretrains_temp

!mv gptsovits_pretrains_temp/* GPT-SoVITS/GPT_SoVITS/pretrained_models/

!rm -rf gptsovits_pretrains_temp


# uvr5 weights

!git clone https://huggingface.co/Delik/uvr5_weights uvr5_weights_temp

!mv uvr5_weights_temp/* GPT-SoVITS/tools/uvr5/uvr5_weights

!rm -rf uvr5_weights_temp


# Chinese ASR models

%cd /content/GPT-SoVITS/tools/asr/models
!git clone https://www.modelscope.cn/damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch.git
!git clone https://www.modelscope.cn/damo/speech_fsmn_vad_zh-cn-16k-common-pytorch.git
!git clone https://www.modelscope.cn/damo/punc_ct-transformer_zh-cn-common-vocab272727-pytorch.git

# English/Japanese ASR should be automatically downloaded.

clear_output()
print("Downloaded!")

In [ ]:
#@title Run UI

#@markdown The first run may take more time as it will download the g2pw model.

#@markdown The type of tunnel you wanna use for seeing the public link, so that if one of them is down, you can use the other one.
Tunnel = "Gradio" #@param ["Gradio", "Ngrok", "Cloudflare", "LocalTunnel", "Horizon"]

#@markdown Also when using Ngrok ,Cloudflare, LocalTunnel, or Horizon, you need to wait for the Local URL to appear, and only after that click on the Public URL which is above. Plus, you have to open UVR in the GPT-SoVITS UI before opening the Public URL.


#@markdown Use the following option **only if you chose Ngrok** as the Tunnel, **You need to be a paid ngrok member** as it needs more than 3 tunnels (there are 4 ports):

#@markdown You can get the Ngrok Tunnel Authtoken here: https://dashboard.ngrok.com/tunnels/authtokens/new.

ngrok_tunnel_authtoken = "" #@param {type:"string"}


#@markdown Use the following option **only if you chose Horizon** as the Tunnel:

#@markdown You can get the Horizon ID here: https://hrzn.run/dashboard/ , login, on the 2nd step, it shows an `hrzn login YOUR_ID`, you need to copy that id.

horizon_id = "" #@param {type:"string"}


from IPython.display import clear_output
import time

%cd /content/GPT-SoVITS/


if Tunnel == "Gradio":
  %env is_share = True
elif Tunnel == "Ngrok":
  %env is_share = False
  !pip install pyngrok
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_tunnel_authtoken)
  gptsovits_tunnel = ngrok.connect(9874, bind_tls=True)
  uvr_tunnel = ngrok.connect(9873, bind_tls=True)
  inference_tunnel = ngrok.connect(9872, bind_tls=True)
  proofreading_tunnel = ngrok.connect(9871, bind_tls=True)
  print("GPT-SoVITS Tunnel Public URL:", gptsovits_tunnel.public_url)
  print("UVR Tunnel Public URL:", uvr_tunnel.public_url)
  print("Inference Tunnel Public URL:", uvr_tunnel.public_url)
  print("Proofreading Tunnel Public URL:", proofreading_tunnel.public_url)
elif Tunnel == "Cloudflare":
  %env is_share = False
  # download cloudflare
  !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb
  !rm -rf nohup.out
  import time
  # Run cloudflare
  # gpt so vits
  !nohup cloudflared tunnel --url localhost:9874 &
  time.sleep(7)
  # uvr
  !nohup cloudflared tunnel --url localhost:9873 &
  time.sleep(7)
  # inference
  !nohup cloudflared tunnel --url localhost:9872 &
  time.sleep(7)
  # proofreading
  !nohup cloudflared tunnel --url localhost:9871 &
  clear_output()
  time.sleep(7)
  # Find and print the Cloudflare URL with a prefix
  cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
  print(f"GPT-SoVITS Tunnel Public URL: {cloudflare_url[0]}")
  print(f"UVR Tunnel Public URL: {cloudflare_url[1]}")
  print(f"Inference Tunnel Public URL: {cloudflare_url[2]}")
  print(f"Proofreading Tunnel Public URL: {cloudflare_url[3]}")
elif Tunnel == "LocalTunnel":
  %env is_share = False
  # install
  !npm install -g localtunnel
  import time
  import urllib
  # run localtunnel
  # gptsovits
  with open('url.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 9874 >> url.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  clear_output()

  print(f"GPT-SoVITS Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  # uvr
  with open('url2.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 9873 >> url2.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url2.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  print(f"UVR Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  # inference
  with open('url3.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 9872 >> url3.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url3.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  print(f"Inference Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  # proofreading
  with open('url4.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 9871 >> url4.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url4.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  print(f"Proofreading Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")


  print(f'LocalTunnels Password: {endpoint_ip}')
elif Tunnel == "Horizon":
  # install
  !npm install -g @hrzn/cli
  # login
  !hrzn login $horizon_id
  # run horizon
  # gptsovits
  with open('url.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('hrzn tunnel http://localhost:9874 >> url.txt 2>&1 &')

  time.sleep(4)

  with open('url.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = !grep -oE "https://[a-zA-Z0-9.-]+\.hrzn\.run" url.txt
      tunnel_url = tunnel_url[0]

  clear_output()

  print(f"GPT-SoVITS Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m")

  # uvr
  with open('url.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('hrzn tunnel http://localhost:9873 >> url2.txt 2>&1 &')

  time.sleep(4)

  with open('url2.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = !grep -oE "https://[a-zA-Z0-9.-]+\.hrzn\.run" url2.txt
      tunnel_url = tunnel_url[0]

  print(f"UVR Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m")

  # inference
  with open('url3.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('hrzn tunnel http://localhost:9872 >> url3.txt 2>&1 &')

  time.sleep(4)

  with open('url3.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = !grep -oE "https://[a-zA-Z0-9.-]+\.hrzn\.run" url3.txt
      tunnel_url = tunnel_url[0]

  print(f"Inference Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m")

  # proofreading
  with open('url4.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('hrzn tunnel http://localhost:9871 >> url4.txt 2>&1 &')

  time.sleep(4)

  with open('url4.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = !grep -oE "https://[a-zA-Z0-9.-]+\.hrzn\.run" url4.txt
      tunnel_url = tunnel_url[0]

  print(f"Proofreading Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m")


import subprocess
!python webui.py